In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 10000
N_bound = 200

# x,t

la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

In [3]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')
t_exact = data['t'].flatten()[:,None]
x_exact = data['x'].flatten()[:,None]
Exact_u = np.real(data['usol'])

In [ ]:
print('t:',t_exact.shape)
print('x:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

In [ ]:
T_exact, X_exact = np.meshgrid(t_exact,x_exact)
print('T_exact:',T_exact.shape)
print('X_exact:',X_exact.shape)

In [6]:
T_exact_flatten = T_exact.flatten()[:,None]
X_exact_flatten = X_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None]  

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(6, 2))

cmap = plt.get_cmap('jet')  
heatmap1 = ax1.imshow(Exact_u, interpolation='nearest', cmap='rainbow',  extent=[0, 1, -1, 1], origin='lower', aspect='auto')
ax1.set_title('u', fontsize=15)
ax1.set_xlabel(r'$t$', fontsize=15)
ax1.set_ylabel(r'$x$', fontsize=15)

# Add a color bar
cbar = fig.colorbar(heatmap1, ax=ax1, fraction=0.15, pad=0.04)
cbar.set_label('Value', fontsize=12)

plt.show()

In [ ]:
random_seed = 1234
np.random.seed(random_seed)
number_observe_data = 100

index_t = (lhs(1,number_observe_data))*len(t_exact)
index_t = np.floor(index_t).reshape(number_observe_data,).astype(int)
observe_t = t_exact[index_t]

index_x = (lhs(1,number_observe_data))*len(x_exact)
index_x = np.floor(index_x).reshape(number_observe_data,).astype(int)
observe_x = x_exact[index_x]

observe_data = np.hstack((observe_x,observe_t))
observe_clear_u = Exact_u[index_x,index_t].reshape(-1,1)

############# N(0,0.1^2) #############
noise_nu = 0
noise_std = 0.1
noise_u = np.random.normal(loc=noise_nu, scale=noise_std, size=observe_clear_u.shape)
observe_u = observe_clear_u + noise_u
############# N(0,0.1^2) #############

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_clear_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

noise_u = numpy_to_tensor(noise_u, var_name="noise_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
np.random.seed(5678)
n_test_data = 10000

index_test_data = (lhs(1,n_test_data))*len(data_star)
index_test_data = np.floor(index_test_data).reshape(n_test_data,).astype(int)

index_t = (lhs(1,n_test_data))*len(t_exact)
index_t = np.floor(index_t).reshape(n_test_data,).astype(int)
test_data_t = t_exact[index_t]

index_x = (lhs(1,n_test_data))*len(x_exact)
index_x = np.floor(index_x).reshape(n_test_data,).astype(int)
test_data_x = x_exact[index_x]

test_data = np.hstack((test_data_x,test_data_t))
test_u = Exact_u[index_x,index_t].reshape(-1,1)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

In [10]:
def output_transform(data_input, u_input):
    
    x_in = data_input[:, 0:1]
    t_in = data_input[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * u_input - torch.sin(np.pi * x_in)

In [11]:
C1 = torch.tensor(0.5, requires_grad=True)

In [12]:
def get_loss_f(x_grad,t_grad,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_grad,t_grad),1)) 
    E_inside = output_transform(torch.cat((x_grad,t_grad),1),E_inside) 
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])
    E_t = compute_higher_order_derivatives(E_inside, [t_grad])


    if return_sequence=='yes':
        return torch.square(E_t+E_inside*E_x-C*(E_xx))
    else:
        return torch.mean(torch.square(E_t+E_inside*E_x-C*(E_xx)))

In [13]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [14]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[20]*8, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')

PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()  

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)
   
optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [15]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [ ]:
Theta_list = np.zeros_like(observe_u.cpu().detach().numpy())
Theta_list = numpy_to_tensor(Theta_list, var_name="Theta_list", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

In [17]:
C1_id = id(C1)
Theta_list_id = id(Theta_list)

optimizer_error = optim.Adam([Theta_list], lr=0.0001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    

In [18]:
def train_optimizer1_error_correction(PINNs,C,Theta_list_p):
    
    for param in PINNs.parameters():
        param.requires_grad = True
    
    C.requires_grad = True
    
    Theta_list_p.requires_grad = False

In [19]:
def train_optimizer2_error_correction(PINNs,C,Theta_list_p):
    
    for param in PINNs.parameters():
        param.requires_grad = False
    
    C.requires_grad = False
    
    Theta_list_p.requires_grad = True

In [20]:
def train_for_error_correction(PINNs,C,Theta_list_input,observe_data_input,observe_u_input):
    
    train_optimizer2_error_correction(PINNs,C,Theta_list_input)
        
    E_observation = PINNs(observe_data_input)
    E_observation = output_transform(observe_data_input,E_observation) 
    
    loss_observation_refine = torch.mean(torch.square(E_observation+Theta_list_input-observe_u_input)) 
    
    optimizer_error.zero_grad()
    loss_observation_refine.backward()
    optimizer_error.step()
        
    train_optimizer1_error_correction(PINNs,C,Theta_list_input)
    
    return Theta_list_input

In [21]:
def flatten_line(data):
    for i in range(1, len(data)):
        if data[i] > data[i - 1]:
            data[i] = data[i - 1]
    return data

In [22]:
import copy
def training_recall(PINNs, C, it_iter, loss_f_for_T_1_best, best_PINNs_state, 
                    loss_all_1, loss_f_1, loss_f_for_collocation_1, 
                    loss_f_for_T_1, loss_f_excapt_T_1, loss_T_1, 
                    loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, test_loss_1, 
                    C1_list, optimizer1):

    loss_f_for_T_1_smooth = flatten_line(loss_f_for_T_1.copy())
    current_min_index = loss_f_for_T_1_smooth.index(min(loss_f_for_T_1_smooth))
    current_min = min(loss_f_for_T_1_smooth)

    if best_PINNs_state is None or current_min < best_PINNs_state['min_loss']:
        best_PINNs_state = {
            'model_state': copy.deepcopy(PINNs.state_dict()),
            'C_state': C.clone().detach(),
            'optimizer_state': copy.deepcopy(optimizer1.state_dict()),
            'it': it_iter,
            'min_loss': current_min
        }

    if len(loss_f_for_T_1_smooth) - current_min_index > 500:
        print('Restoring best model from iteration:', best_PINNs_state['it'])
        PINNs.load_state_dict(best_PINNs_state['model_state'])
        
        C_value = best_PINNs_state['C_state']
        with torch.no_grad(): 
            C.copy_(C_value)  

        optimizer1.load_state_dict(best_PINNs_state['optimizer_state'])
        
        recall_it = it_iter  
        it_iter = best_PINNs_state['it']  

        loss_all_1 = loss_all_1[:it_iter+1]
        loss_f_1 = loss_f_1[:it_iter+1]
        loss_f_for_collocation_1 = loss_f_for_collocation_1[:it_iter+1]
        loss_f_for_T_1 = loss_f_for_T_1[:it_iter+1]
        loss_f_excapt_T_1 = loss_f_excapt_T_1[:it_iter+1]
        loss_T_1 = loss_T_1[:it_iter+1]
        loss_T_clear_1 = loss_T_clear_1[:it_iter+1]
        loss_T_1_test_data = loss_T_1_test_data[:it_iter+1]
        test_loss_1 = test_loss_1[:it_iter+1]
        C1_list = C1_list[:it_iter+1]
        loss_f_for_T_1_best = True
    
    return (PINNs, C, it_iter, loss_f_for_T_1_best, best_PINNs_state, 
            loss_all_1, loss_f_1, loss_f_for_collocation_1, loss_f_for_T_1, 
            loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, 
            test_loss_1, C1_list,optimizer1)

In [23]:
import numpy as np

class AdaptiveChangeLROnPlateau:
    def __init__(self, optimizer, patience_base=10, threshold_ratio = 1e-6, factor_down=0.1, patience_down=10, cooldown_down = 5, factor_up=1.1, patience_up=10, cooldown_up = 5, min_lr=0, max_lr=1, eps=1e-8):
        self.optimizer = optimizer
        self.patience_base = patience_base
        self.factor_down = factor_down
        self.patience_down = patience_down
        self.cooldown_down = cooldown_down
        self.factor_up = factor_up
        self.patience_up = patience_up
        self.cooldown_up = cooldown_up
        
        self.threshold_ratio = threshold_ratio
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.eps = eps
        self.cooldown_counter = 0
        
        self.best_metrics = None
        self.best_base_loss = None
        
        self.num_bad_epochs_base = 0
        self.num_bad_epochs_up = 0
        self.num_bad_epochs_down = 0
        self.renew_iter = 0
        
    def step(self, loss_f_for_T_1_input, loss_f_for_collocation_1_input, to_iter):
        
        loss_f_for_T_1_input = np.array(loss_f_for_T_1_input[self.renew_iter:])
        loss_f_for_collocation_1_input =  np.array(loss_f_for_collocation_1_input[self.renew_iter:])
        
        metrics = (loss_f_for_T_1_input)[-1]-flatten_line(loss_f_for_collocation_1_input)[-1] 
        base_loss = flatten_line(loss_f_for_T_1_input)[-1]    
        
        # Performance improvement logic
        if (self.best_metrics is not None) and (self.best_base_loss is not None):

            if base_loss == self.best_base_loss:
                self.num_bad_epochs_base += 1
                
                if self.num_bad_epochs_base > self.patience_base:
                    self.num_bad_epochs_base = 0
                    self.num_bad_epochs_up = 0
                    self.num_bad_epochs_down = 0
                    self.renew_iter = to_iter
                    
                    for param_group in self.optimizer.param_groups:
                        param_group['lr'] = self.max_lr

                    self._adjust_C_lr(self.factor_down**2, "down")

            if metrics  >=  self.best_metrics + self.threshold_ratio:
                self.num_bad_epochs_up += 1
                
                if self.num_bad_epochs_up > self.patience_up:
                    if self.cooldown_counter > 0:
                        self.cooldown_counter -= 1
                    else:
                        self._adjust_lr(self.factor_up, "up")
                        
                        self.cooldown_counter = self.cooldown_up
                        self.num_bad_epochs_base = 0
                        self.num_bad_epochs_up = 0
                        self.num_bad_epochs_down = 0
                        
            elif metrics  < self.best_metrics - self.threshold_ratio:
                self.num_bad_epochs_down += 1
                
                if self.num_bad_epochs_down > self.patience_down:
                    if self.cooldown_counter > 0:
                        self.cooldown_counter -= 1
                    else:
                        self._adjust_lr(self.factor_down, "down")
                        
                        self.cooldown_counter = self.cooldown_down
                        self.num_bad_epochs_base = 0
                        self.num_bad_epochs_up = 0
                        self.num_bad_epochs_down = 0

        self.best_metrics = metrics
        self.best_base_loss = base_loss
        
    def _adjust_lr(self, factor, direction):
        
        for param_group in self.optimizer.param_groups:
            old_lr = param_group['lr']
            
            if old_lr == 0:
                old_lr = self.min_lr
                
            if direction == "down":
                new_lr = old_lr * factor
                if new_lr<self.min_lr:
                    new_lr = 0
            else:
                new_lr = min(old_lr * factor, self.max_lr)
                
            param_group['lr'] = new_lr

    def _adjust_C_lr(self, factor, direction):
        
        for param_group in optimizer1.param_groups:
            if any(id(param) == C1_id for param in param_group['params']):
                old_lr = param_group['lr']

                if old_lr == 0:
                    old_lr = self.min_lr

                if direction == "down":
                    new_lr = old_lr * factor
    
                    if new_lr<=self.min_lr:
                        new_lr = 0

                param_group['lr'] = new_lr  

In [24]:
scheduler = AdaptiveChangeLROnPlateau(optimizer_error, patience_base=100, threshold_ratio = 1e-6,
                                      factor_down=0.1, patience_down=50, cooldown_down = 0, 
                                      factor_up=10, patience_up=50, cooldown_up = 0,
                                      min_lr=1e-8, max_lr=1e-4, eps=1e-12)

In [ ]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
loss_T_EC_1 = []
test_loss_1 = []
C1_list = []
############## Record list ###############

nIter1 = 20000
it = 0


# Initial placeholders for best model and iteration
loss_f_for_T_1_best = False
best_PINNs_state = None
best_it = None
recall_it = None

while it<nIter1:
   
    #########loss f #########
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1, return_sequence='not')
    
    #########loss f  for collocation data#########
    loss_f_for_collocation = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  for observation data#########
    loss_f_for_T = get_loss_f(observe_data_x_inside,observe_data_t_inside,PINNs1,C1,return_sequence='not')
    #########loss f  excapt observation data#########
    loss_f_excapt_T = get_loss_f(test_data_x_inside,test_data_t_inside,PINNs1,C1, return_sequence='not')
    
    #########loss T observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation) 
    
    if loss_f_for_T_1_best:
        loss_observation = torch.mean(torch.square(E_observation+Theta_list-observe_u)) 
    else:
        loss_observation = torch.mean(torch.square(E_observation-observe_u))            
    
    #########loss EC######### 
    loss_EC = torch.mean(torch.square(Theta_list-noise_u))
    
    #########loss T noisy observation#########  
    loss_observation_noisy = torch.mean(torch.square(E_observation-observe_u)) 
    
    #########loss T clear observation#########        
    E_observation_clear = PINNs1(observe_data) 
    E_observation_clear = output_transform(observe_data,E_observation_clear) 
    loss_observation_clear = torch.mean(torch.square(E_observation_clear-observe_clear_u))        
    
    #########loss T excapt observation#########        
    E_observation_excapt = PINNs1(test_data) 
    E_observation_excapt = output_transform(test_data,E_observation_excapt) 
    loss_observation_excapt = torch.mean(torch.square(E_observation_excapt-test_u)) 
    
    #########loss PI#########
    loss = loss_f+10*loss_observation

    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u) 
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    loss_f_1.append(loss_f.item())
    loss_f_for_collocation_1.append(loss_f_for_collocation.item())
    loss_f_for_T_1.append(loss_f_for_T.item())
    loss_f_excapt_T_1.append(loss_f_excapt_T.item())
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation_noisy.item()) 
    loss_T_clear_1.append(loss_observation_clear.item()) 
    loss_T_1_test_data.append(loss_observation_excapt.item()) 
    loss_T_EC_1.append(loss_EC.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
    
    if it % 1000 == 0:
        
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        
        for param_group in optimizer_error.param_groups:
            if any(id(param) == Theta_list_id for param in param_group['params']):
                Theta_list_lr = param_group['lr']
                print(f'Current Theta_list learning rate: {Theta_list_lr}')
                
        for param_group in optimizer1.param_groups:
            if any(id(param) == C1_id for param in param_group['params']):
                C1_lr = param_group['lr']
                print(f'Current C1 learning rate: {C1_lr}')
                
        print('C1',C1)  
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()

    if  not loss_f_for_T_1_best:
        if it>3000:
            
            (PINNs1, C1, it, loss_f_for_T_1_best, best_PINNs_state, 
             loss_all_1, loss_f_1, loss_f_for_collocation_1, loss_f_for_T_1, 
             loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, loss_T_1_test_data,loss_T_EC_1, 
             test_loss_1, C1_list, optimizer1) = training_recall(
                 PINNs1, C1, it, loss_f_for_T_1_best, best_PINNs_state, 
                 loss_all_1, loss_f_1, loss_f_for_collocation_1, 
                 loss_f_for_T_1, loss_f_excapt_T_1, loss_T_1, loss_T_clear_1, 
                 loss_T_1_test_data,loss_T_EC_1, test_loss_1, C1_list, optimizer1)
            
            if loss_f_for_T_1_best:
                Theta_list.requires_grad = True
                for param_group in optimizer1.param_groups:
                    if any(id(param) == C1_id for param in param_group['params']):
                        param_group['lr'] = 0.0001  
    else:
        ############################ Refine L_T ############################   
        scheduler.step(loss_f_for_T_1,loss_f_for_collocation_1,it)
              
        Theta_list = train_for_error_correction(PINNs1,C1,Theta_list,observe_data,observe_u)
        ############################ Refine L_T ############################  

    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)